<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #12: Analyse Zooniverse classifications</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Sept 29th, 2021</h5>

# Set up and requirements

### Import Python packages

In [15]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import utils.t12_utils as t12
from utils.zooniverse_utils import retrieve_zoo_info, populate_subjects, populate_agg_annotations
import getpass

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Choose your project

In [16]:
project = t12.choose_project()

Dropdown(description='Project:', options=('Koster Seafloor Obs', 'Spyfish Aotearoa', 'SGU'), value='Koster Sea…

### Initiate SQL database and populate sites, movies and species

In [17]:
# Specify the path of the movies 
movies_path = "/uploads"

# Specify the path of the sql database
db_path = "koster_lab.db"

# Initiate the SQL database 
%run -i "../db_starter/starter.py" --movies_path $movies_path --db_path $db_path --project_name project.value

Updated sites
Updated movies
Updated species


### Retrieve Zooniverse information

In [18]:
# Save your user name, password and Zooniverse project number.
zoo_user = getpass.getpass('Enter your Zooniverse user')
zoo_pass = getpass.getpass('Enter your Zooniverse password')

Enter your Zooniverse user········
Enter your Zooniverse password········


In [19]:
# Specify the Zooniverse information required throughout the tutorial
zoo_info = ["subjects", "workflows", "classifications"]

# Retrieve and store the Zooniverse information required throughout the tutorial in a dictionary
project_n, zoo_info_dict = retrieve_zoo_info(zoo_user, zoo_pass, project.value, zoo_info)

Connecting to the Zooniverse project
Retrieving subjects from Zooniverse
subjects were retrieved successfully
Retrieving workflows from Zooniverse
workflows were retrieved successfully
Retrieving classifications from Zooniverse
classifications were retrieved successfully


In [20]:
# Populate the sql with subjects uploaded to Zooniverse
populate_subjects(zoo_info_dict["subjects"], project.value, db_path)

Updated subjects
The database has a total of 2342 frame subjects and 7362 clip subjects have been updated


### Step 1: Specify the Zooniverse workflow id and version of interest

*Note:  A manual export in Zooniverse is required to get the most up-to-date classifications here**

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

In [21]:
# Display a selectable list of workflow names and the type of subject
workflows_df = zoo_info_dict["workflows"]
workflow_name, subj_type = t12.choose_workflows(workflows_df)

Dropdown(description='Workflow name:', options=('Species identification', 'test ', 'Species location (intermed…

Dropdown(description='Subject type:', index=1, options=('frame', 'clip'), value='clip')

In [73]:
# Selects the workflow id based on the workflow name
workflow_id = workflows_df[workflows_df.display_name==workflow_name.value].workflow_id.unique()[0]

# Display a selectable list of versions of the workflow of interest
workflow_version = t12.choose_w_version(workflows_df, workflow_id)

Dropdown(description='Minimum workflow version:', options=(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, …

In [74]:
# Retrieve classifications from the workflow of interest
class_df = t12.get_classifications(workflow_id,
                                   workflow_version.value, 
                                   subj_type.value, 
                                   zoo_info_dict["classifications"],
                                   db_path)

Zooniverse classifications have been retrieved


### Step 2: Aggregate classifications received on the workflow of interest

In [75]:
# Specify the agreement threshold required among cit scientists
agg_params = t12.choose_agg_parameters(subj_type.value)

FloatSlider(value=0.8, continuous_update=False, description='Aggregation threshold:', max=1.0, readout_format=…

IntSlider(value=3, continuous_update=False, description='Min numbers of users:', max=15, min=1, style=SliderSt…

FloatSlider(value=0.8, continuous_update=False, description='Object threshold:', max=1.0, readout_format='.1f'…

FloatSlider(value=0.5, continuous_update=False, description='IOU Epsilon:', max=1.0, readout_format='.1f', sty…

FloatSlider(value=0.8, continuous_update=False, description='Inter user agreement:', max=1.0, readout_format='…

In [76]:
agg_class_df = t12.aggregrate_classifications(class_df, subj_type.value, project.value, agg_params)

Aggregrating the classifications
1804 classifications aggregated out of 2858 unique subjects available


In [58]:
#TODO move this cell to ML and upload frames tutorials
# Populate the sql with the aggregated classifications
#populate_agg_annotations(agg_class_df, subj_type.value, db_path)

Updated agg_annotations_frame


### Step 3: Summarise the number of classifications based on the agreement specified

In [60]:
agg_class_df.groupby("label")["subject_id"].agg("count")

,subject_id
label,


### Step 4: Display the aggregated classifications in a table

In [71]:
# Display the dataframe into a table
agg_class_df[["subject_id","label","how_many","first_seen"]]

KeyError: "['how_many', 'first_seen'] not in index"

### Step 5: Use the subject explorer widget to get more information about specific subjects and their classifications

In [ ]:
# Launch the subject viewer
t12.launch_viewer(agg_class_df)

In [ ]:
# END